In [51]:
#!/usr/bin/env python
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

In [52]:
from __future__ import print_function
import sys
import time

from openvino.inference_engine import IENetwork, IEPlugin

In [53]:
def colorize(y, ycrcb):
    y[y>255] = 255
    y[y<0] = 0

    img = np.zeros((y.shape[0], y.shape[1], 3), np.uint8)
    img[:,:,0] = y
    img[:,:,1] = ycrcb[:,:,1]
    img[:,:,2] = ycrcb[:,:,2]
    img = cv2.cvtColor(img, cv2.COLOR_YCR_CB2RGB)
    
    return img

# PSNR measure, from ANR's code
def PSNR(pred, gt):
    f = pred.astype(float)
    g = gt.astype(float)
    e = (f - g).flatten()
    n = len(e)
    rst = 10*np.log10(n/e.dot(e))
    
    return rst

In [54]:
model_xml = "/home/yue/caffe-rc5/examples/SRCNN/SRCNN_iter_11112465.xml"
model_bin = os.path.splitext(model_xml)[0] + ".bin"
prob_threshold = 0.5
labels_map = None

In [55]:
print("Initializing plugin for CPU device...")
plugin = IEPlugin(device="CPU", plugin_dirs=None)

Initializing plugin for CPU device...


In [56]:
print("Adding CPU extenstions...")
plugin.add_cpu_extension("/opt/intel/computer_vision_sdk/deployment_tools/inference_engine/lib/ubuntu_16.04/intel64/libcpu_extension_sse4.so")

Adding CPU extenstions...


In [57]:
print("Reading IR...")
net = IENetwork.from_ir(model=model_xml, weights=model_bin)
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))

Reading IR...


In [58]:
print("Loading IR to the plugin...")
exec_net = plugin.load(network=net, num_requests=2)
# Read and pre-process input image
n, c, h, w = net.inputs[input_blob]

Loading IR to the plugin...


In [59]:
input_stream = "/home/yue/caffe-rc5/examples/SRCNN/Test/videos/dance.mp4"
cap = cv2.VideoCapture(input_stream)
cur_request_id = 0
next_request_id = 1

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

render_time = 0
total_time = 0
fps = 0

In [60]:
while cap.isOpened():
        total_start = time.time()
        ret, frame = cap.read()
        if not ret:
            break
        
        initial_w = cap.get(3)
        initial_h = cap.get(4)
        
        im_raw = cv2.resize(frame, (w, h))

        im_resize = im_raw.transpose((2, 0, 1))  # Change data layout from HWC to CHW

        im_resize = cv2.resize(im_raw, (w, h))
        ycrcb = cv2.cvtColor(im_resize, cv2.COLOR_RGB2YCR_CB)

        #print('ycrcb.shape: ', ycrcb.shape)
        im_resize = ycrcb[:,:,0]
        im_resize = im_resize.reshape((im_resize.shape[0], im_resize.shape[1], 1))
        #print('im_resize.shape: ', im_resize.shape)
        im_input = im_resize

        im_input = im_input.reshape((n, c, h, w))
        #print('im_input.shape: ', im_input.shape)
        infer_start = time.time()
        res = exec_net.infer(inputs={input_blob: im_input})
        det_time = time.time() - infer_start
        mat = res['conv3']
        mat = np.squeeze(mat, axis=0)
        ycrcb = ycrcb[6:-6,6:-6,:]
        im_pred = colorize(mat[0,:,:], ycrcb)
        im_pred = cv2.resize(im_pred, (im_raw.shape[1],im_raw.shape[0]))
        #################################################################################################################
        # Draw performance stats

        inf_time_message = "Inference time: {:.3f} ms".format(det_time * 1000)
        render_time_message = "OpenCV rendering time: {:.3f} ms".format(render_time * 1000)
        total_time_message = "Total Time Taken per Frame: {:.3f} ms".format(total_time * 1000)
        fps_message = "FPS: {:.1f}".format(fps);

        cv2.putText(im_pred, inf_time_message, (15, 15), cv2.FONT_HERSHEY_COMPLEX, 0.5, (200, 10, 10), 1)
        cv2.putText(im_pred, render_time_message, (15, 30), cv2.FONT_HERSHEY_COMPLEX, 0.5, (10, 10, 200), 1)
        cv2.putText(im_pred, total_time_message, (15, 45), cv2.FONT_HERSHEY_COMPLEX, 0.5, (10, 10, 200), 1)
        cv2.putText(im_pred, fps_message, (15, 60), cv2.FONT_HERSHEY_COMPLEX, 0.5, (10, 10, 200), 1)
        
        
        render_start = time.time()
        cv2.imshow("SR", im_pred)
        cv2.imshow("Original", im_raw)
        render_end = time.time()
        
        render_time = render_end - render_start
        total_time = render_end - total_start
        
        fps = float((1/total_time))
        
        key = cv2.waitKey(1)
        if key == 27:
            break
            
cv2.destroyAllWindows()
del plugin